In [18]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset , DataLoader
import torch.nn.functional as F

In [3]:
def generate_synthetic_data(filename,num_students = 1000):
  np.random.seed(182)
  hours_studied = np.random.uniform(1,10,num_students)
  attendence = np.random.uniform(50,100,num_students)
  previous_score = np.random.uniform(40,90,num_students)
  final_score = (1.44*hours_studied + 2.71 * attendence + 7.32 * previous_score + np.random.normal(0,5,num_students)) % 100
  df = pd.DataFrame({
      "hours_studied" : hours_studied,
      "attendence" : attendence,
      "previous_score" : previous_score,
      "final_score" : final_score
  })
  df.to_csv(filename,index = False)

generate_synthetic_data("student_data.csv",1500)

In [4]:
def generate_synthetic_data(filename, num_people = 10000):
  np.random.seed(611)
  weight = np.random.uniform(50,140,num_people)
  height = np.random.uniform(1.5, 2.1, num_people)
  bmi = (weight/(height**2))
  df = pd.DataFrame({
      "weights" : weight,
      "heights" : height,
      "BMI" : bmi
  })
  df.to_csv(filename,index=False)

generate_synthetic_data("people_bmi.csv",5000)

In [5]:
df = pd.read_csv("people_bmi.csv")
df.head()

,weights,heights,BMI
0,77.567777,1.694229,27.023218
1,58.753574,1.622462,22.319543
2,105.982453,1.500373,47.079894
3,99.104250,2.041272,23.784312
4,105.522623,1.614064,40.504593


In [41]:
class BMI(Dataset):
    def __init__(self, csv_path):
        super(BMI, self).__init__()
        df = pd.read_csv(csv_path)

        # Separate features and labels
        features = df[['weights', 'heights']].values
        labels = df['BMI'].values

        # Normalize features (important!)
        self.feature_mean = features.mean(axis=0)
        self.feature_std = features.std(axis=0)
        features = (features - self.feature_mean) / self.feature_std

        # Store as float32
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [42]:
# Load dataset and split into train/test
dataset = BMI("people_bmi.csv")
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
features, label = next(iter(train_loader))
print(f"Features : {features} \n Label : {label}")

Features : tensor([[ 75.7003,   1.9660],
        [ 51.4367,   1.9456],
        [133.3232,   1.6008],
        [ 85.1726,   1.6017],
        [101.8391,   1.8409],
        [138.8738,   1.6459],
        [132.3478,   1.6053],
        [106.9438,   1.8538],
        [ 66.7456,   1.8267],
        [ 90.3936,   1.5059],
        [ 81.2205,   1.9473],
        [131.0703,   2.0778],
        [ 65.8871,   1.5647],
        [114.7265,   1.6254],
        [ 70.1760,   1.6293],
        [ 71.7194,   1.6778],
        [ 78.0464,   1.7393],
        [139.0037,   2.0649],
        [ 71.6636,   1.7559],
        [ 67.1664,   1.8783],
        [113.8786,   1.9413],
        [124.5976,   1.5627],
        [100.5564,   2.0855],
        [126.0202,   1.9269],
        [132.0864,   1.8283],
        [131.9500,   1.5120],
        [125.8689,   1.8877],
        [ 81.0901,   1.5612],
        [120.3168,   1.9507],
        [ 94.7260,   2.0412],
        [ 88.8935,   1.5575],
        [117.0162,   1.7282],
        [ 64.0325,   1.9581],

In [55]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Conv1d(2, 16) # Increased number of neurons
    self.fc2 = nn.Conv1d(16, 32) # Added more layers
    self.fc3 = nn.Conv1d(32, 16)
    self.fc4 = nn.Conv1d(16, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x) # Removed ReLU from the last layer
    return x

In [56]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)  # Changed to Adam with lr=0.001
criterion = nn.MSELoss()

TypeError: Conv1d.__init__() missing 1 required positional argument: 'kernel_size'

In [44]:
# Training function (simplified output)
def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        target = target.view(-1, 1)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}: Train Loss = {avg_loss:.4f}")

In [45]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.view(-1, 1)
            output = model(data)
            test_loss += criterion(output, target).item()

    test_loss /= len(test_loader)
    print(f"Test Loss = {test_loss:.4f}\n")

In [57]:
num_epochs = 150
for epoch in range(num_epochs):
    train(model, device, train_loader, criterion, optimizer, epoch)
    if epoch % 5 == 0:  # Test every 5 epochs
        test(model, device, test_loader, criterion)

Epoch 0: Train Loss = 0.0055
Test Loss = 0.0042

Epoch 1: Train Loss = 0.0054
Epoch 2: Train Loss = 0.0057
Epoch 3: Train Loss = 0.0049
Epoch 4: Train Loss = 0.0052
Epoch 5: Train Loss = 0.0059
Test Loss = 0.0059

Epoch 6: Train Loss = 0.0050
Epoch 7: Train Loss = 0.0060
Epoch 8: Train Loss = 0.0057
Epoch 9: Train Loss = 0.0049
Epoch 10: Train Loss = 0.0061
Test Loss = 0.0067

Epoch 11: Train Loss = 0.0055
Epoch 12: Train Loss = 0.0061
Epoch 13: Train Loss = 0.0051
Epoch 14: Train Loss = 0.0051
Epoch 15: Train Loss = 0.0053
Test Loss = 0.0041

Epoch 16: Train Loss = 0.0056
Epoch 17: Train Loss = 0.0054
Epoch 18: Train Loss = 0.0051
Epoch 19: Train Loss = 0.0058
Epoch 20: Train Loss = 0.0050
Test Loss = 0.0092

Epoch 21: Train Loss = 0.0053
Epoch 22: Train Loss = 0.0047
Epoch 23: Train Loss = 0.0052
Epoch 24: Train Loss = 0.0049
Epoch 25: Train Loss = 0.0054
Test Loss = 0.0093

Epoch 26: Train Loss = 0.0060
Epoch 27: Train Loss = 0.0057
Epoch 28: Train Loss = 0.0049
Epoch 29: Train Loss

In [60]:
# Test prediction with normalized input
weight_test = 80.0
height_test = 1.80
x = torch.tensor([[weight_test, height_test]], dtype=torch.float32)

In [59]:
# Normalize using the same stats as training (ensure float32)
x_normalized = (x - torch.tensor(dataset.feature_mean, dtype=torch.float32)) / torch.tensor(dataset.feature_std, dtype=torch.float32)
output = model(x_normalized)
predicted_bmi = output.item()
actual_bmi = weight_test / (height_test ** 2)

In [54]:
print(f"\nTest Prediction:")
print(f"Weight: {weight_test} kg, Height: {height_test} m")
print(f"Predicted BMI: {predicted_bmi:.2f}")
print(f"Actual BMI: {actual_bmi:.2f}")


Test Prediction:
Weight: 80.0 kg, Height: 1.8 m
Predicted BMI: 24.72
Actual BMI: 24.69
